# Metrics, validation strategies and baselines

В данном jupyter notebook рассматриваются примеры того, какие схемы валидации и метрики используются в рекомендательных системах.
Также построим простые модели (бейзлайны) на данных МТС Библиотеки. 

* [Preprocessing](#preprocessing)
* [General remarks](#general-remarks)
* [Metrics](#metrics)
    * [Regression](#regression)
    * [Classification](#classification)
    * [Ranking](#ranking)
* [Validation strategies](#validation)
* [Baselines](#baselines)

In [1]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise



<a id="preprocessing"></a>
# Preprocessing

Загрузим наши данные, теперь уже с фичами, и применим знания из [pandas-scipy-for-recsys](https://www.kaggle.com/sharthz23/pandas-scipy-for-recsys)

In [2]:
df = pd.read_pickle('./interactions_preprocessed.pickle')
df_users = pd.read_pickle('./users_preprocessed.pickle')
df_items = pd.read_pickle('./items_preprocessed.pickle')

In [3]:
test_dates = df['start_date'].unique()[-7:]



In [4]:
test_dates = list(pairwise(test_dates))
test_dates

[(numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (numpy.datetime64('2019-12-26T00:00:00.000000000'),
  numpy.datetime64('2019-12-27T00:00:00.000000000')),
 (numpy.datetime64('2019-12-27T00:00:00.000000000'),
  numpy.datetime64('2019-12-28T00:00:00.000000000')),
 (numpy.datetime64('2019-12-28T00:00:00.000000000'),
  numpy.datetime64('2019-12-29T00:00:00.000000000')),
 (numpy.datetime64('2019-12-29T00:00:00.000000000'),
  numpy.datetime64('2019-12-30T00:00:00.000000000')),
 (numpy.datetime64('2019-12-30T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000'))]

In [5]:
split_dates = test_dates[0]

In [6]:
train = df[df['start_date'] < split_dates[0]]

In [7]:
test = df[(df['start_date'] >= split_dates[0]) & (df['start_date'] < split_dates[1])]

In [8]:
test = test[(test['rating'] >= 4) | (test['rating'].isnull())]

In [9]:
split_dates, train.shape, test.shape


((numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (1517994, 5),
 (2114, 5))

<a id="baselines"></a>
# Baselines

Будем строить бейзлайн по популярному в зависимости от возраста пользователя

In [10]:
df_users

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0
...,...,...,...
142883,159606,25_34,0.0
142884,159607,25_34,NaN
142885,159609,18_24,0.0
142886,159610,35_44,0.0


In [11]:
train.head(2)

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01


In [12]:
test.head(2)

,user_id,item_id,progress,rating,start_date
1517914,38753,135245,0,NaN,2019-12-25
1517915,101642,319500,67,5.0,2019-12-25


In [32]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date' , df_u = df_users):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        self.min_date = None
        self.df_u = df_u
        self.df_res = None

    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.min_date = min_date
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values

    def super_fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.min_date = min_date
        self.df_res = df.loc[df[self.dt_column] > min_date, self.item_column]
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values


    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [33]:
pop_model = PopularRecommender(days=77, dt_column='start_date')

In [27]:
vars(pop_model)

{'max_K': 100,
 'days': 77,
 'item_column': 'item_id',
 'dt_column': 'start_date',
 'recommendations': [],
 'min_date': None,
 'df_u':         user_id     age  sex
 0             1   45_54  NaN
 1             2   18_24  0.0
 2             3  65_inf  0.0
 3             4   18_24  0.0
 4             5   35_44  0.0
 ...         ...     ...  ...
 142883   159606   25_34  0.0
 142884   159607   25_34  NaN
 142885   159609   18_24  0.0
 142886   159610   35_44  0.0
 142887   159611   35_44  0.0
 
 [142888 rows x 3 columns]}

In [34]:
pop_model.super_fit(train)

In [35]:
vars(pop_model)

{'max_K': 100,
 'days': 77,
 'item_column': 'item_id',
 'dt_column': 'start_date',
 'recommendations': array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
        270415, 285394, 218025, 115190, 147734, 282647, 208935, 271846,
         74650, 281005, 168037, 135032, 168900, 219099, 288531,  75579,
        226196,  96052,   9197, 169853, 312940, 103077,  99616,  63978,
          6136, 311394,  86588,  43622, 295007, 110811, 104010, 176898,
        210979,  48510, 107522,  55913,  67643, 181955,  30772, 141961,
        233762, 126581, 247959,  93249, 238200,  39878, 276498, 238856,
        303990,  86123,   5001,   8875,  31454, 241026,  28889, 209326,
        265984,  26963, 231923, 211217, 243711, 121687,  90674, 108759,
        232317, 163618, 251856,  11424, 233644,  58803, 240571,  49352,
        231257,  12301, 297013, 145016,  65912,  99357, 181857, 226128,
         81827,  35226, 121544, 206185, 312792,  90225,  34363, 223806,
        222102, 250772,  25173, 22

In [41]:
pop_model.df_res.value_counts().head(10).index.values

array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
       270415, 285394], dtype=int64)

In [ ]:
dt_column = 'start_date'
item_column = 'item_id'
df.loc[df[dt_column] > min_date, item_column]

In [21]:
train.head(2)

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01


In [23]:
vars(pop_model)

{'max_K': 100,
 'days': 77,
 'item_column': 'item_id',
 'dt_column': 'start_date',
 'recommendations': array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
        270415, 285394, 218025, 115190, 147734, 282647, 208935, 271846,
         74650, 281005, 168037, 135032, 168900, 219099, 288531,  75579,
        226196,  96052,   9197, 169853, 312940, 103077,  99616,  63978,
          6136, 311394,  86588,  43622, 295007, 110811, 104010, 176898,
        210979,  48510, 107522,  55913,  67643, 181955,  30772, 141961,
        233762, 126581, 247959,  93249, 238200,  39878, 276498, 238856,
        303990,  86123,   5001,   8875,  31454, 241026,  28889, 209326,
        265984,  26963, 231923, 211217, 243711, 121687,  90674, 108759,
        232317, 163618, 251856,  11424, 233644,  58803, 240571,  49352,
        231257,  12301, 297013, 145016,  65912,  99357, 181857, 226128,
         81827,  35226, 121544, 206185, 312792,  90225,  34363, 223806,
        222102, 250772,  25173, 22

In [24]:
df_users

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0
...,...,...,...
142883,159606,25_34,0.0
142884,159607,25_34,NaN
142885,159609,18_24,0.0
142886,159610,35_44,0.0


In [ ]:
pop_model.recommendations

In [ ]:
pop_model.days

In [ ]:
item_titles = dict(zip(df_items.id,df_items.title))

In [ ]:
item_titles[128115]

In [ ]:
print(item_titles.get(128115))

In [ ]:
top10_recs = pop_model.recommend()
top10_recs

In [ ]:
map(item_titles.get, top10_recs)

In [ ]:
list(map(item_titles.get, top10_recs))

In [ ]:
# create datframe with user_id fo recs
recs = pd.DataFrame({'user_id': test['user_id'].unique()})


In [ ]:
recs.head(2)

In [ ]:

top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

In [ ]:
recs = recs.explode('item_id')
recs.head(top_N + 2)

In [ ]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

In [ ]:
recs.rename(columns={'rank':'predict_rank'} , inplace = True)

In [ ]:
recs.head(2)

In [ ]:
test.head(2)

In [ ]:

test_recs = test.merge(recs, how = 'left' , on = ['user_id','item_id'])

In [ ]:
test_recs.head(2)

In [ ]:
test_recs['users_item_count'] = test_recs.groupby(level='user_id', sort=False)['predict_rank'].transform(np.size)

In [ ]:
uic = test_recs.groupby('user_id').agg({'item_id':'count'}).reset_index().rename(columns = {'item_id':'users_item_count'})

In [ ]:
test_recs = test_recs.merge(uic , how = 'left' , on = 'user_id')

In [ ]:
test_recs['pred_reciprocal_rank'] = 1 / test_recs['predict_rank']
test_recs['pred_reciprocal_rank'] = test_recs['pred_reciprocal_rank'].fillna(0)

In [ ]:
test_recs.head(2)

In [ ]:
test_recs = test_recs.sort_values(by=['user_id', 'predict_rank'])



In [ ]:
test_recs['cumulative_rank'] = test_recs.groupby('user_id').cumcount() + 1

In [ ]:
test_recs

In [ ]:
test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['predict_rank']

In [ ]:
test_recs.head(2)

In [ ]:
test_recs[test_recs['predict_rank'].notnull()]

In [ ]:
print(f'Метрик по test ({str(split_dates[0])[:10]}, {str(split_dates[1])[:10]})')

In [ ]:
users_count = test_recs['user_id'].nunique()

In [ ]:
users_count

In [ ]:
for k in range(1, top_N + 1):
    hit_k = f'hit@{k}'
    test_recs[hit_k] = test_recs['predict_rank'] <= k
    print(f'Precision@{k} = {(test_recs[hit_k] / k).sum() / users_count:.4f}')
    print(f"Recall@{k} = {(test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count:.4f}")

In [ ]:
mapN = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
print(f"MAP@{top_N} = {mapN}")

In [ ]:
mrr = test_recs.groupby('user_id').agg({'pred_reciprocal_rank':'max'}).reset_index()['pred_reciprocal_rank'].mean()
print(f"MRR = {mrr}")